In [1]:
import requests

resp = requests.get("https://db.ygoprodeck.com/api/v7/cardinfo.php")
cards_json = resp.content

In [2]:
import json
deserialized = json.loads(cards_json)

In [3]:
from tqdm import tqdm
common_cards = list() 
for card in tqdm(deserialized["data"]):
    if "card_sets" in card:
        available_rarities = set([card_set["set_rarity"] for card_set in card["card_sets"]])
        if "Common" in available_rarities:
            authorized = True
            if "banlist_info" in card:
                if "ban_tcg" in card["banlist_info"]:
                    authorized = "Banned" == card["banlist_info"]["ban_tcg"]
            authorized = card["type"] not in ["Skill Card", "Token"]
            if authorized:
                common_cards.append(card)


100%|██████████| 12878/12878 [00:00<00:00, 236827.73it/s]


In [4]:
import csv
with open("common_cards.tsv", "w+") as csvfile:
    writer = csv.writer(csvfile, delimiter="\t", lineterminator="\n")
    writer.writerow(["name", "card_type", "type", "attribute", "atk", "def", "level", "scale", "number_authorized"])
    for card in tqdm(common_cards):
        name = card["name"]
        card_type = card["type"]
        type = card["race"]
        if "attribute" in card:
            attribute = card["attribute"]
            atk = card["atk"]
        else:
            attribute = "N/A"
            atk = "N/A"
        if "def" in card:
            defense = card["def"]
            level = card["level"]
        else:
            defense = "N/A"
            level = "N/A"
        if "scale" in card:
            scale = card["scale"]
        else:
            scale = "N/A"
        if "banlist_info" in card:
            if "ban_tcg" in card["banlist_info"]:
                if card["banlist_info"]["ban_tcg"] =="Limited":
                    number_authorized = 1
                elif card["banlist_info"]["ban_tcg"] == "Semi-Limited":
                    number_authorized = 2
                elif card["banlist_info"]["ban_tcg"] == "Banned":
                    number_authorized = 0
            else:
                number_authorized = 3
        else:
            number_authorized = 3
                
        writer.writerow([name, card_type, type, attribute, atk, defense, level, scale, number_authorized])


100%|██████████| 7694/7694 [00:00<00:00, 85718.55it/s]


{'ban_ocg': 'Banned'}